In [1]:
#import libraries
import pandas as pd
import sqlite3
#import openpyxl

In [3]:
# Import each sheet, first row as column headers
products = pd.read_excel('data/sales original.xlsx', sheet_name='products', header=0)
stock = pd.read_excel('data/sales original.xlsx', sheet_name='stock', header=0)
assessment = pd.read_excel('data/sales original.xlsx', sheet_name='assessment', header=0)
invoices = pd.read_excel('data/sales original.xlsx', sheet_name='invoices', header=0)
customers = pd.read_excel('data/sales original.xlsx', sheet_name='customers', header=0)

### Check the tables

In [22]:
stock.nunique()

StockCode    3942
ASIN         4134
dtype: int64

Okay, this is a problem. Apparently there are different ASIN with the same StockCode.<br />
Question, are these just typos or do these products share the same physical stock space? Even if they share the same space, they should not have the same StockCode. Would make it difficult to rearrange just one of them or update number in stock.

In [23]:
# Show the different unique ASINs associated with a StockCode
out = stock.astype(str).groupby('StockCode').agg(lambda x: ', '.join(x.unique()))

# but only those that contain more than one ASIN
out = out.loc[out['ASIN'].str.len() > 10]
out#.count()

,ASIN
StockCode,
10080,"B07D9D82R7, B000L47AHG"
10133,"B074VB8VT5, B07CM7N7TG"
15058A,"B0189228CS, B07D263CHY"
15058C,"B07NQMD216, B07D263CHY"
16008,"B016NDCXKM, B000L47AHG"
...,...
90195A,"B07JCKZ3RC, B000L47AHG"
90210D,"B07L9B57G5, B000L47AHG"
DCGS0003,"B07DNV7GJL, B07PQ8S46W"


657 StockCodes share two or more ASIN. Will this impact my choice of primary key? Don't think so. The combination of ASIN+StockCode would still be unique to a product. Only difficulty I see is if two products with a Null ASIN share the same StockCode but have a different title.<br />
Let's check that.

In [24]:
products1 = products.copy()

# Extract StockCode and insert into test copy of products
extracted_col = stock['StockCode']
products1.insert(1, 'StockCode', extracted_col)

# Extract all the products with a Null ASIN
null_products = products1.loc[products1['ASIN'] == 'Null']

In [25]:
# Show all products where the title is Null
null_products.loc[null_products.title == 'Null']

,ASIN,StockCode,title,product_type


In [26]:
null_products.nunique()

ASIN             1
StockCode       70
title           64
product_type     5
dtype: int64

In the list of products with a Null ASIN all have a title. That is good. There are alo no products with no or a Null title.<br />
But there are 70 different StockCodes and only 64 different titles. So some StockCodes are associated with the same title. That is not optimal but better than the other way around. This way the combination of ASIN+StockCode as a primary key will still work.

Assessment will be merged into the products table, because records were appended sequentially this is not problematic. If assessment contained distinct review records instead of aggregated values for each product the assessment table would have been kept.

The customers table contains a primary key already, but let's check for null values and if there are CustomerIDs that point to different countries.

In [27]:
customers.loc[customers.CustomerID == 'Null']

,CustomerID,Country
11,Null,Germany
12,Null,Germany
13,Null,Germany
14,Null,Germany
15,Null,Germany
...,...,...
554375,Null,Germany
554376,Null,Germany
554377,Null,Germany
554378,Null,Germany


There are 139315 records in which the CustomerID is not known. This is not good. This means that invoices have a Null foreign key in place of the CustomerID and the invoice cannot be assigned to a customer.<br />
I could resolve this by merging the tables invoices and customers, grouping by InvoiceNo and assigning a new value to CustomerID where Null appears in an invoice. Running the risk of assigning a new ID to an existing customer.

Are there any invoices that do not have an ID?

In [28]:
invoices.loc[invoices.InvoiceNo == 'Null']

,InvoiceNo,ASIN,Quantity,price,total_sale,invoice_date,invoice_time,CustomerID


Thankfully there are no invoices that do not have an ID or are empty. Therefore customerIDs where there is a Null can be assigned a new ID based on an invoice.<br />
But there are of course null ASINs in the invoices which cannot be assigned to a product.

### Modifications

In [29]:
# In products: insert StockCode after ASIN to create a combined primary key
extracted_col = stock['StockCode']
products.insert(1, 'StockCode', extracted_col)

# insert price of a single item
extracted_col = invoices['price']
products.insert(4, 'price', extracted_col)

In [30]:
# In products: insert assessment; since there are no distinct reviews (with reviews) but only aggregated ones, it does not
# make sense to keep them in a separate table
products['rating'] = pd.Series(assessment['rating'])
products['review_count'] = pd.Series(assessment['review_count'])

In [31]:
products.columns

Index(['ASIN', 'StockCode', 'title', 'product_type', 'price', 'rating',
       'review_count'],
      dtype='object')

In [32]:
# Create a copy of invoices as a linking table to create a many-to-many relationship 
# between products and invoice.
# Could probably do this with slice too
invoice_product = invoices.copy()

# In invoice_product: insert StockCode and product_type after ASIN to create a combined foreign key
extracted_col = stock['StockCode']
invoice_product.insert(2, 'StockCode', extracted_col)
extracted_col = products['product_type']
invoice_product.insert(3, 'product_type', extracted_col)

# Delete not needed columns
invoice_product = invoice_product.drop(['invoice_date', 'invoice_time', 'CustomerID'], axis=1)

In [33]:
# Delete columns from invoices now in invoice_products
invoices = invoices.drop(['ASIN', 'Quantity', 'price', 'total_sale'], axis=1)


In [34]:
# Transfer the customers table over to the invoice table
invoices['CustomerID'] = pd.Series(customers['CustomerID'])
invoices['Country'] = pd.Series(customers['Country'])

Begin: Testing the customerID assignment

In [35]:
invoices1 = invoices.copy()

In [36]:
# Show me the invoices that do not have a CustomerID
null_invoice = invoices1.loc[invoices1.CustomerID == 'Null']

In [37]:
# Assign 9+last four digits of invoice number to CustomerID == Null
# That way we can keep the invoices with Null Customers and assign a "temporary" ID to Null Customers
for index, row in invoices1.iterrows():
    if row['CustomerID'] == 'Null':
        invoices1.at[index, 'CustomerID'] = '9' + str(row['InvoiceNo'])[-4:]


In [38]:
# Check and compare null_invoice to the newly assigned
new_df = invoices1.loc[invoices1['CustomerID'].str.startswith('9', na=False)].copy()
new_df.groupby(['InvoiceNo', 'CustomerID']).size().reset_index().rename(columns={0:'count'})

,InvoiceNo,CustomerID,count
0,78536544,96544,551
1,78536555,96555,2
2,78536558,96558,1
3,78536565,96565,2
4,78536592,96592,618
...,...,...,...
2045,78581497,91497,60
2046,78581498,91498,233
2047,78A563185,93185,1
2048,78A563186,93186,1


In [39]:
null_invoice.groupby(['InvoiceNo', 'CustomerID']).size().reset_index().rename(columns={0:'count'})

,InvoiceNo,CustomerID,count
0,78536544,Null,551
1,78536555,Null,2
2,78536558,Null,1
3,78536565,Null,2
4,78536592,Null,618
...,...,...,...
2045,78581497,Null,60
2046,78581498,Null,233
2047,78A563185,Null,1
2048,78A563186,Null,1


End: Testing the customerID assignment

In [40]:
# Assign temporary IDs to Null CustomerIDs in invoices with an initial 9
# Why the trouble? Each invoice with a Null customerID can now be associated with an
# individual Null customer instead of lumping all Nulls together (n.b. will screw up 
# your country analysis). If that customer already had an ID or has another invoice 
# as Null is irrelevant. This way we do not have to discard the Null records in invoice
# and customers.
for index, row in invoices.iterrows():
    if row['CustomerID'] == 'Null':
        invoices.at[index, 'CustomerID'] = '9' + str(row['InvoiceNo'])[-4:]

In [41]:
# Transfer the customers back over to the customer table
customers['CustomerID'] = pd.Series(invoices['CustomerID'])
customers['Country'] = pd.Series(invoices['Country'])

# And drop the customers' country in the invoices table
invoices = invoices.drop(['Country'], axis=1)

In [42]:
# in stock: drop ASIN, delete duplicates
# in assessment: drop entire dataframe
# in products: delete duplicates
# in customers: delete duplicates
# in invoices: delete duplicates
# in invoice_product: delete duplicates, delete price

In [43]:
# Need this for counting the countries of each duplicate CustomerID later
customers_orig = customers.copy()

In [44]:
stock = stock.drop(['ASIN'], axis=1)
stock = stock.drop_duplicates()

products = products.drop_duplicates()

customers = customers.drop_duplicates()

invoices = invoices.drop_duplicates()

invoice_product = invoice_product.drop_duplicates()
invoice_product = invoice_product.drop(['price'], axis=1)

Check for duplicates in what will be the primary keys.<br />
There are 18 duplicates in the customers table, 10 of which are temporary 9-er IDs, meaning some of the InvoiceNo either ended in the same 4 digits or the Country is different for that ID.

In [45]:
#print(customers[customers['CustomerID'].duplicated() == True])

# Compare the 18 duplicates
pd.concat(g for _, g in customers.groupby('CustomerID') if len(g) > 1)

,CustomerID,Country
29852,12370,Cyprus
35444,12370,Austria
180899,12394,Belgium
392660,12394,Denmark
35415,12417,Belgium
174672,12417,Spain
64973,12422,Australia
244572,12422,Switzerland
20537,12429,Denmark
171442,12429,Austria


What to do?

In [46]:
print(customers['Country'].value_counts())

Germany                 5788
United Kingdom           103
France                    90
Ireland                   31
Spain                     30
Belgium                   25
Switzerland               24
Portugal                  20
Italy                     14
Finland                   12
Austria                   11
Norway                    10
Hong Kong                 10
Unknown                    9
Netherlands                9
Australia                  9
Denmark                    9
Sweden                     8
Cyprus                     8
Japan                      8
Israel                     6
Poland                     6
Canada                     4
Greece                     4
United States              4
Bahrain                    3
Malta                      2
United Arab Emirates       2
Saudi Arabia               1
Czech Republic             1
Iceland                    1
Lebanon                    1
Brazil                     1
Singapore                  1
European Commu

There are some countries where the loss of one instance in customers would hugely impact the statistics.<br />
Unless I think of something better, I'll have to go through all 18 and see which country has the largest count, i.e. in the original data, for CustomerID 99999 how many rows are Country x versus Country y, and use the one with the most instances and drop the other.

We are using the customers_orig dataframe to count the number of instances of a country (copied before removal of duplicates in customers).

In [47]:
# print(customers_orig.Country[customers_orig['CustomerID'] == 12370].value_counts())
# Cyprus     166
# Austria      9

# print(customers_orig.Country[customers_orig['CustomerID'] == 12394].value_counts())
# Belgium    20
# Denmark     6

# print(customers_orig.Country[customers_orig['CustomerID'] == 12417].value_counts())
# Belgium    166
# Spain       23

# print(customers_orig.Country[customers_orig['CustomerID'] == 12422].value_counts())
# Australia      22
# Switzerland    17

# print(customers_orig.Country[customers_orig['CustomerID'] == 12429].value_counts())
# Denmark    76
# Austria    20

# print(customers_orig.Country[customers_orig['CustomerID'] == 12431].value_counts())
# Australia    191
# Belgium       53

# print(customers_orig.Country[customers_orig['CustomerID'] == 12455].value_counts())
# Spain     48
# Cyprus    46

# print(customers_orig.Country[customers_orig['CustomerID'] == 12457].value_counts())
# Switzerland    59
# Cyprus          2

#------
# I have no idea why the temporary IDs generated above (starting with 9...) require 
# quotation marks to work.
#------

# print(customers_orig.Country[customers_orig['CustomerID'] == '91652'].value_counts())
# Hong Kong    29
# Germany       2

# print(customers_orig.Country[customers_orig['CustomerID'] == '92540'].value_counts())
# Germany    181
# Ireland     10

# print(customers_orig.Country[customers_orig['CustomerID'] == '92541'].value_counts())
# Germany    185
# Ireland      2

# print(customers_orig.Country[customers_orig['CustomerID'] == '93547'].value_counts())
# Ireland    23
# Germany     1

# print(customers_orig.Country[customers_orig['CustomerID'] == '95927'].value_counts())
# Ireland    65
# Germany     2

# print(customers_orig.Country[customers_orig['CustomerID'] == '95928'].value_counts())
# Ireland    46
# Germany     1

# print(customers_orig.Country[customers_orig['CustomerID'] == '96558'].value_counts())
# Germany      1
# Hong Kong    1

# print(customers_orig.Country[customers_orig['CustomerID'] == '97675'].value_counts())
# Hong Kong    15
# Germany       1

# print(customers_orig.Country[customers_orig['CustomerID'] == '99337'].value_counts())
# Germany    307
# Ireland     29

# print(customers_orig.Country[customers_orig['CustomerID'] == '99738'].value_counts())
# Germany      96
# Hong Kong    35


In [48]:
# Dropping the CustomerId+Country combination where in the original data for the CustomerID the country has the least
# number of instances.
customers.drop(customers[(customers['CustomerID'] == 12370) & (customers['Country'] == 'Austria')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == 12394) & (customers['Country'] == 'Denmark')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == 12417) & (customers['Country'] == 'Spain')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == 12422) & (customers['Country'] == 'Switzerland')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == 12429) & (customers['Country'] == 'Austria')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == 12431) & (customers['Country'] == 'Belgium')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == 12457) & (customers['Country'] == 'Cyprus')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == '91652') & (customers['Country'] == 'Germany')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == '92540') & (customers['Country'] == 'Ireland')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == '92541') & (customers['Country'] == 'Ireland')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == '93547') & (customers['Country'] == 'Germany')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == '95927') & (customers['Country'] == 'Germany')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == '95928') & (customers['Country'] == 'Germany')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == '97675') & (customers['Country'] == 'Germany')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == '99337') & (customers['Country'] == 'Ireland')].index, inplace = True)
customers.drop(customers[(customers['CustomerID'] == '99738') & (customers['Country'] == 'Hong Kong')].index, inplace = True)

# One CustomerID+Country combinations had equal numbers (96558) and one only had a difference of 2 (12455)
# In these two cases both records were deleted, as the most likely country could not be determined
customers.drop(customers[(customers['CustomerID'] == '96558') | (customers['CustomerID'] == 12455)].index, inplace = True)


There is one invoice that has a duplicate InvoiceNo. It belongs to an original CustomerID.<br />
This did not delete when dropping duplicates because the invoice time is once 11 and once 12. Maybe the order went through just as the hour went from 11 to 12. No way to know without the minutes and seconds or a proper timestamp.<br />
I'm dropping the one containing 12 manually, because there are more four times as many rows associated with 11 than 12.

In [49]:
print(invoices[invoices['InvoiceNo'].duplicated() == True])

       InvoiceNo invoice_date  invoice_time CustomerID
151777  78549245   2019-04-07            12      15005


In [50]:
invoices.drop(invoices[(invoices['InvoiceNo'] == 78549245) & (invoices['invoice_time'] == 12)].index, inplace = True)

There are 186 products that have the same combination of ASIN+StockCode.

In [51]:
print(products.duplicated(subset=['ASIN','StockCode']).sum())

186


ASIN B07KGHFB8N : is listed both as a keyboard and a mouse<br />
ASIN B07Y693ND1 : ditto

In [52]:
print(products[products.duplicated(subset=['ASIN','StockCode']) == True])

              ASIN StockCode  \
1485    B07KGHFB8N     84744   
1491    B07Y693ND1     20725   
1497    B00FNKMVUO     21232   
1498    B007X3VEX4     21258   
1501    B07JJGCSHR     21429   
...            ...       ...   
522780  B07JWHFM2B     20711   
543359  B07T8DH3CG     21109   
546296  B07RPVG1PD     21688   
546297  B07RPVG1PD     21693   
546724  B07YBS9PGN     85104   

                                                    title product_type  \
1485    Razer Holiday Bundle 2018 Cynosa Chroma Gaming...        mouse   
1491    Razer DeathAdder Essential Gaming Mouse: 6400 ...        mouse   
1497    UtechSmart Venus Gaming Mouse RGB Wired, 16400...        mouse   
1498    iHome Bluetooth Mac Mouse with Scroll Wheel, 3...        mouse   
1501    Wireless Keyboard and Mouse, Vssoplor 2.4GHz R...        mouse   
...                                                   ...          ...   
522780  Comidox Ergonomic Mousepad with Wrist Support ...        mouse   
543359  Mini Wireless K

Okay, how about ASIN+StockCode+product_type as Primary Key?

In [53]:
print(products.duplicated(subset=['ASIN','StockCode','product_type']).sum())

0


This is rather a long Primary Key but would work if we did not want to add an additional individual key.

Write the dataframes stock, products, customers, invoices, invoice_product to a database.<br />
First try SQLite and then try mySQL.

In [54]:
# Create a file sales_database.db in the data directory [of what? SQLite? Jupyter?]
# This file is an empty SQLite database with sales_db pointing to the database.
sales_db = sqlite3.connect("sales_database.db")

In [55]:
# Establishing a cursor object that executes SQL code against the database
c = sales_db.cursor()

In [56]:
stock.columns

Index(['StockCode'], dtype='object')

In [57]:
customers.columns

Index(['CustomerID', 'Country'], dtype='object')

In [58]:
invoices.columns

Index(['InvoiceNo', 'invoice_date', 'invoice_time', 'CustomerID'], dtype='object')

In [59]:
invoice_product.columns

Index(['InvoiceNo', 'ASIN', 'StockCode', 'product_type', 'Quantity',
       'total_sale'],
      dtype='object')

In [60]:
products.columns

Index(['ASIN', 'StockCode', 'title', 'product_type', 'price', 'rating',
       'review_count'],
      dtype='object')

### Create and populate an SQLite database

In [ ]:
# Create empty tables in the database sales_database.db
c.execute(
    """
    CREATE TABLE stock (
        StockCode TEXT,
        PRIMARY KEY(StockCode)
        );
    """
)

c.execute(
    """
    CREATE TABLE customers (
        CustomerID INTEGER,
        Country TEXT,
        PRIMARY KEY(CustomerID)
        );
    """
)

c.execute(
    """
    CREATE TABLE invoices (
        InvoiceNo TEXT,
        invoice_date TEXT,
        invoice_time INTEGER,
        CustomerID INTEGER,
        PRIMARY KEY(InvoiceNo),
        FOREIGN KEY(CustomerID) REFERENCES customers(CustomerID)
        );
    """
)

c.execute(
    """
    CREATE TABLE invoice_product (
        InvoiceNo TEXT,
        ASIN TEXT,
        StockCode TEXT,
        product_type TEXT,
        Quantity INTEGER,
        total_sale REAL,
        FOREIGN KEY(ASIN, StockCode, product_type) REFERENCES products(ASIN, StockCode, product_type),
        FOREIGN KEY(InvoiceNo) REFERENCES invoices(InvoiceNo)
        );
    """
)

c.execute(
    """
    CREATE TABLE products (
        ASIN TEXT,
        StockCode TEXT,
        title TEXT,
        product_type TEXT,
        price REAL,
        rating REAL,
        review_count INTEGER,
        PRIMARY KEY(ASIN, StockCode, product_type),
        FOREIGN KEY(StockCode) REFERENCES stock(StockCode)
        );
    """
)


In [ ]:
# populate
stock.to_sql('stock', sales_db, if_exists='append', index=False)
invoices.to_sql('invoices', sales_db, if_exists='append', index=False)
customers.to_sql('customers', sales_db, if_exists='append', index=False)
products.to_sql('products', sales_db, if_exists='append', index=False)
invoice_product.to_sql('invoice_product', sales_db, if_exists='append', index=False)

In [45]:
# close connection
sales_db = sqlite3.connect("sales_database.db")

### Create and populate a mySQL database

Discontinued because of unresolved permissions issues. Will try again at a later time.